In [ ]:
!pip install tweet-preprocessor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import preprocessor as p
import re
import warnings
warnings.filterwarnings("ignore")
from bs4 import BeautifulSoup
import string
import html
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import nltk
from nltk import word_tokenize, FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import xgboost as xgb
from google.colab import files

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
uploaded = files.upload()

Saving sentiment_analysis.csv to sentiment_analysis (3).csv


In [ ]:
df = pd.read_csv('sentiment_analysis.csv')
df = df.drop(['ID'], axis=1)
df.head()

,text,label
0,Josh Jenkins is looking forward to TAB Breeder...,1
1,RT @MianUsmanJaved: Congratulations Pakistan o...,1
2,"RT @PEPalerts: This September, @YESmag is taki...",1
3,"RT @david_gaibis: Newly painted walls, thanks ...",1
4,RT @CedricFeschotte: Excited to announce: as o...,1


In [ ]:
def clean_text(_sentence):

    #removes emojis, pictographs, map symbols and flags
    emoji_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F"
          u"\U0001F300-\U0001F5FF"
          u"\U0001F680-\U0001F6FF"
          u"\U0001F1E0-\U0001F1FF"
                            "]+", flags = re.UNICODE)
    _sentence = emoji_pattern.sub(r'',_sentence)

    #emoves tags and attributes
    _sentence = re.sub('(<[^>]*>)','',_sentence)

    #converts character code to string
    _sentence = html.unescape(_sentence)

    #removes urls
    _sentence = re.sub("(\w+:\/\/\S+)|(http[^\s]+)|(www.[^\s]+)",'',_sentence, flags=re.MULTILINE)

    #removes RT
    _sentence = re.sub("RT",'',_sentence)

    #converts the sentence to a lower case
    _sentence = _sentence.lower()

    #removes punctuation
    _sentence = re.sub(r'\@\w+|\#\w+','', _sentence)
    _sentence = _sentence.translate(str.maketrans('', '', (string.punctuation)))

    #checks for stop words
    stop_words = set(stopwords.words('english'))
    _sentence = [word for word in str(_sentence).split() if word not in stop_words]

    #joins the word and returns a sentence
    return " ".join(_sentence)

#function to preprocess tweets
def preprocess_tweets(tweets_df):

    #apply clean text in a vectorized format
    vfunc_clean_text = np.vectorize(clean_text)
    tweets_df['text'] = vfunc_clean_text(tweets_df['text']) #create a new column that consists of preprocessed tweets

    return tweets_df

In [ ]:
df = preprocess_tweets(df)

In [ ]:
df

,text,label
0,josh jenkins looking forward tab breeders crow...,1
1,congratulations pakistan becoming world odds,1
2,september taking maine mendoza’s surprise than...,1
3,newly painted walls thanks million custodial p...,1
4,excited announce july 2017 feschotte lab reloc...,1
...,...,...
550386,cant stop watching hermm,0
550387,poor old tom odell doesnt look like would know...,0
550388,smashed 7 ants awesome gamehjfjfi,1
550389,morning girls wonderful,1


In [ ]:
X = df.loc[:, ['text']]
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=1)

In [ ]:
vectorizer = CountVectorizer(binary=False,ngram_range=(1,1))
X_train_ = vectorizer.fit_transform(X_train['text'])
X_test_ = vectorizer.transform(X_test['text'])
print (X_train.shape, X_test.shape)

(440312, 1) (110079, 1)


TV = TfidfVectorizer(max_features = 1000)   
X_train = TV.fit_transform(X_train['text'])
X_test = TV.fit_transform(X_test['text'])
print(np.shape(X_train))
print(np.shape(X_test))

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
def model_training(model, X_train, y_train, X_test, y_test):
    model.fit(X_train,y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)

    acc_train = metrics.accuracy_score(y_train, y_pred_train)
    acc_test = metrics.accuracy_score(y_test, y_pred_test)

    #Print Accuracies for comparison
    print("The accuracy score for the Train set is: {}".format(acc_train))
    print("The accuracy score for the Test set is: {}".format(acc_test))
    print(classification_report(y_test, y_pred_test))
    print(confusion_matrix(y_test, y_pred_test, labels=[0, 1]))



In [ ]:
KNN_model = KNeighborsClassifier()
print("Results when KNN model is used:\n")
model_training(KNN_model, X_train_, y_train,  X_test_, y_test)

In [ ]:
LR_model = LogisticRegression()
print("Results when Logistic Regression is used:\n")
model_training(LR_model, X_train_, y_train,  X_test_, y_test)

Results when Logistic Regression is used:

The accuracy score for the Train set is: 0.9834617271389379
The accuracy score for the Test set is: 0.973310077308115
              precision    recall  f1-score   support

           0       0.96      0.96      0.96     35810
           1       0.98      0.98      0.98     74269

    accuracy                           0.97    110079
   macro avg       0.97      0.97      0.97    110079
weighted avg       0.97      0.97      0.97    110079

[[34457  1353]
 [ 1585 72684]]


In [ ]:
SVC_model = LinearSVC()
print("Results when SVC model is used:\n")
model_training(SVC_model, X_train_, y_train,  X_test_, y_test)

Results when SVC model is used:

The accuracy score for the Train set is: 0.9910472573992987
The accuracy score for the Test set is: 0.971429609643983
              precision    recall  f1-score   support

           0       0.95      0.96      0.96     35810
           1       0.98      0.98      0.98     74269

    accuracy                           0.97    110079
   macro avg       0.97      0.97      0.97    110079
weighted avg       0.97      0.97      0.97    110079

[[34348  1462]
 [ 1683 72586]]


In [ ]:
MultinomialNB_model = MultinomialNB()
print("Results when Naive Bayes model is used:\n")
model_training(MultinomialNB_model, X_train_, y_train,  X_test_, y_test)

Results when Naive Bayes model is used:

The accuracy score for the Train set is: 0.964629626265012
The accuracy score for the Test set is: 0.9534243588695391
              precision    recall  f1-score   support

           0       0.94      0.91      0.93     35810
           1       0.96      0.97      0.97     74269

    accuracy                           0.95    110079
   macro avg       0.95      0.94      0.95    110079
weighted avg       0.95      0.95      0.95    110079

[[32681  3129]
 [ 1998 72271]]


In [ ]:
randomforest_model = RandomForestClassifier()
print("Results when Random Forest model is used:\n")
model_training(randomforest_model, X_train_, y_train,  X_test_, y_test)

Results when Random Forest model is used:



In [ ]:
decisiontree_model = DecisionTreeClassifier()
print("Results when Decision Tree model is used:\n")
model_training(decisiontree_model, X_train_, y_train,  X_test_, y_test)

Logistic

In [ ]:
len(X_train['text'].tolist())

440312

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
bag = count.fit_transform(X_train['text'].tolist())

count.vocabulary_

{'try': 127454,
 'new': 87426,
 'chicken': 28054,
 'nugget': 89439,
 'dipped': 38163,
 'favorite': 46744,
 'dipping': 38168,
 'sauce': 108708,
 'taste': 121769,
 'great': 54838,
 'enjoy': 43153,
 'latest': 72774,
 'album': 10490,
 'getting': 52516,
 'books': 21782,
 'literacy': 74932,
 'lol': 75520,
 'things': 123774,
 'get': 52483,
 'hilarious': 59485,
 'involved': 64821,
 'ignore': 62550,
 'wait': 132892,
 'faux': 46712,
 'outrage': 92516,
 'hypocrisy': 61928,
 'bite': 20398,
 'arse': 13947,
 'brazilian': 22707,
 'soccer': 115227,
 'team': 122123,
 'plane': 96954,
 'crashes': 32779,
 'colombia': 30313,
 '76': 6725,
 'dead': 35754,
 'noah': 88415,
 'exceeded': 44803,
 'expectations': 45152,
 'please': 97209,
 'donate': 39346,
 'ly': 77034,
 'wish': 135673,
 'video': 131781,
 '11yr': 1160,
 'old': 90778,
 'us': 130373,
 'cracking': 32688,
 'clarinet': 29141,
 'cases': 26270,
 'making': 77983,
 'fart': 46528,
 'noises': 88526,
 'ugh': 128750,
 'make': 77922,
 'guide': 55615,
 'stupid': 

In [ ]:
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)

# Feed the tf-idf transformer with our previously created Bag of Words
tfidf.fit_transform(bag).toarray()

In [ ]:
from collections import Counter

vocab = Counter()
for twit in X_train.SentimentText:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

In [ ]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

In [ ]:
import math

def plot_distribution(vocabulary):

    hist, edges = np.histogram(list(map(lambda x:math.log(x[1]),vocabulary.most_common())), density=True, bins=500)

    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="Word distribution accross all twits")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555", )
    show(p)

plot_distribution(vocab)

In [ ]:
import nltk

nltk.download('stopwords')

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

plot_distribution(vocab_reduced)

In [ ]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [ ]:
# Note: This may take a long while to execute, like... 1 or 2 hours
gs_lr_tfidf.fit(X_train, y_train)

In [ ]:
print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

In [ ]:
clf = gs_lr_tfidf.best_estimator_
print('Accuracy in test: %.3f' % clf.score(X_test, y_test))

In [ ]:
import pickle
import os

pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)

In [ ]:
twits = [
    "This is really bad, I don't like it at all",
    "I love this!",
    ":)",
    "I'm sad... :("
]

preds = clf.predict(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> {preds[i]}')

KNN

In [ ]:
# Finding all the unique words in training data's Tweet column
train_unique =(list(set(X_train['text'].str.findall("\w+").sum())))
train_unique_words = len(train_unique)

In [ ]:
#Training Data: Extracting features and storing them into the training feature matrix
train_matrix = []
for sentence in X_train['text']:
  train_featurevec = []
  word = sentence.split()
  for w in train_unique:
    train_featurevec.append(word.count(w))
  train_matrix.append(train_featurevec)

In [ ]:
X_train = trainfeatures #train_matrix - 2D feature vector
X_test = testfeatures #test_matrix - 2D feature vector
y_train = trained.iloc[:,0].values #trained['Sentiment]
y_test = test.iloc[:,0].values #test['Sentiment]

for k in [1,3,5,7,10]:

  classifier = KNeighborsClassifier(n_neighbors=k,algorithm='brute') #Using brute-force algorithm for quicker computation.
  classifier.fit(X_train, y_train) #Fitting the built-in sklearn classifier on our training data
  predicted_label = classifier.predict(X_test) #Making the classifier to predict on the previously unseen test data.

  accuracy_score = (metrics.accuracy_score(y_test,predicted_label))
  accuracy_score = (round(accuracy_score,2))*100
  acc_list.append(accuracy_score)

  confusion_mat = confusion_matrix(y_test, predicted_label)
  class_report = classification_report(y_test, predicted_label)

  macro_precision = (metrics.precision_score(y_test, predicted_label, average='macro'))
  macro_precision = (round(macro_precision,2))*100
  prec_list.append(macro_precision)

  macro_recall = (metrics.recall_score(y_test, predicted_label, average='macro'))
  macro_recall = (round(macro_recall,2))*100
  rec_list.append(macro_recall)

  macro_f1 = (metrics.f1_score(y_test, predicted_label, average='macro'))
  macro_f1 = (round(macro_f1,2))*100
  f1_list.append(macro_f1)

  print("\n\nConfusion Matrix for k = {} is:\n".format(k))
  print(confusion_mat)
  print("\nClassification Report for k = {} is:\n".format(k))
  print(class_report)


Here is the code for XGBoost classification

In [ ]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Convert text column to list
X_train1_text = X_train1['text'].tolist()
X_test1_text = X_test1['text'].tolist()

# Fit CountVectorizer and transform text data
vectorizer = CountVectorizer(binary=False, ngram_range=(1, 1))
X_train1_ = vectorizer.fit_transform(X_train1_text)
X_test1_ = vectorizer.transform(X_test1_text)

# Create DMatrix object with categorical feature enabled
dtrain = xgb.DMatrix(data=X_train1_, label=y_train1, enable_categorical=True)
dtest = xgb.DMatrix(data=X_test1_, enable_categorical=True)

# Train XGBoost model
params = {'max_depth': 3, 'eta': 0.1, 'objective': 'binary:logistic'}
xgb_model = xgb.train(params=params, dtrain=dtrain)

# Make predictions
y_predictions_train = xgb_model.predict(dtrain)
y_predictions_test = xgb_model.predict(dtest)

# Convert predicted probabilities to binary values using a threshold of 0.5
y_pred_train = (y_predictions_train > 0.5).astype(int)
y_pred_test = (y_predictions_test > 0.5).astype(int)

# Calculate accuracies
acc_train = metrics.accuracy_score(y_train1, y_pred_train, normalize=True, sample_weight=None)
acc_test = metrics.accuracy_score(y_test1, y_pred_test, normalize=True, sample_weight=None)

# Print accuracies and other metrics
print("The accuracy score for the Train set is: {}".format(acc_train))
print("The accuracy score for the Test set is: {}".format(acc_test))
print(classification_report(y_test, y_pred_test))
print(confusion_matrix(y_test, y_pred_test, labels=[0, 1]))



The accuracy score for the Train set is: 0.6749009793055833
The accuracy score for the Test set is: 0.674951625650669
              precision    recall  f1-score   support

           0       0.33      0.00      0.00     35810
           1       0.67      1.00      0.81     74269

    accuracy                           0.67    110079
   macro avg       0.50      0.50      0.40    110079
weighted avg       0.56      0.67      0.54    110079

[[   17 35793]
 [   34 74235]]
